# Optimization Tutorial

The best current example for how to use SymForce for optimization from Python is the tutorial example in `README.md`.

For examples of how to run optimization from C++, see `symforce/examples/bundle_adjustment_in_the_large` and `symforce/examples/bundle_adjustment`.